In [1]:
import numpy as np
np.set_printoptions(threshold=np.inf)  
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
import matplotlib.pyplot as plt

In [2]:
#Crawler for holiday info of HK using bs4 (Observe the data from HK by oberving the date of peak speed)
import re
import urllib
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
def craw_web_page_hk(url_):
    site = url_
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site, headers=hdr)
    response = urlopen(req)
    # response = urllib.request.urlopen(url_)
    contents = response.read().decode('utf-8')
    soup = BeautifulSoup(contents, 'html.parser')
    #print(soup.get_text)
    tables = soup.find_all("tbody")

    returnHolidy = []
    n = 0
    is_first = True
    for i in range(1,len(tables[0].contents),2):
        if is_first:
            is_first = False
            continue

        data_tr = tables[0].contents[i]
        td_trs = data_tr.contents
        returnHolidy.append([])
        returnHolidy[n].append(td_trs[3].string)
        n = n + 1
    return False, returnHolidy

if __name__ == "__main__":

    print("Hong Kong General holidays for 2017")
    ho2017 = craw_web_page_hk("https://www.gov.hk/tc/about/abouthk/holiday/2017.htm")
    print(ho2017)
    
    print("Hong Kong General holidays for 2018")
    ho2018 = craw_web_page_hk("https://www.gov.hk/tc/about/abouthk/holiday/2018.htm")
    print(ho2018)
    
ho2017 = ho2017[1]
ho2018 = ho2018[1]
for i in range(0,len(ho2017)):
    ho2017[i] = ho2017[i][0]
    ho2018[i] = ho2018[i][0]


Hong Kong General holidays for 2017
(False, [['1月2日'], ['1月28日'], ['1月30日'], ['1月31日'], ['4月4日'], ['4月14日'], ['4月15日'], ['4月17日'], ['5月1日'], ['5月3日'], ['5月30日'], ['7月1日'], ['10月2日'], ['10月5日'], ['10月28日'], ['12月25日'], ['12月26日']])
Hong Kong General holidays for 2018
(False, [['1月1日'], ['2月16日'], ['2月17日'], ['2月19日'], ['3月30日'], ['3月31日'], ['4月2日'], ['4月5日'], ['5月1日'], ['5月22日'], ['6月18日'], ['7月2日'], ['9月25日'], ['10月1日'], ['10月17日'], ['12月25日'], ['12月26日']])


In [3]:
#add weather, find weather through API from https://www.worldweatheronline.com/developer/
#but finally give up using it
weather = pd.read_csv('hongkong.csv')
#data preprocessing
weather['day'] = weather['date_time'].map(lambda x:x.split(' ')[0])
weather['year'] = weather['date_time'].map(lambda x:x.split('-')[0])
weather['month'] = weather['date_time'].map(lambda x:x.split('-')[1])
weather['day'] = weather['date_time'].map(lambda x:x.split('-')[2])
weather['day'] = weather['day'].map(lambda x:x.split(' ')[0])
weather['hour'] = weather['date_time'].map(lambda x:x.split(' ')[1])
weather['hour'] = weather['hour'].map(lambda x:x.split(':')[0])

weather.drop("location",axis=1,inplace=True)
weather.drop("date_time",axis=1,inplace=True)

#good behaviour
# weather.drop("sunrise",axis=1,inplace=True) #处理
# weather.drop("sunset",axis=1,inplace=True) #处理

#bad behaviour
weather.drop("precipMM",axis=1,inplace=True)
weather.drop("DewPointC",axis=1,inplace=True)
weather.drop("moon_illumination",axis=1,inplace=True)
weather.drop("moonrise",axis=1,inplace=True)
weather.drop("moonset",axis=1,inplace=True)
weather.drop("totalSnow_cm",axis=1,inplace=True)
weather.drop("cloudcover",axis=1,inplace=True)
weather.drop("WindGustKmph",axis=1,inplace=True)
weather.drop("windspeedKmph",axis=1,inplace=True)
weather.drop("winddirDegree",axis=1,inplace=True)
weather.drop("visibility",axis=1,inplace=True)
weather.drop("tempC",axis=1,inplace=True)
weather.drop("sunHour",axis=1,inplace=True)
weather.drop("WindChillC",axis=1,inplace=True)
weather.drop("humidity",axis=1,inplace=True)
weather.drop("uvIndex",axis=1,inplace=True)
weather.drop("maxtempC",axis=1,inplace=True)
weather.drop("mintempC",axis=1,inplace=True)
weather.drop("HeatIndexC",axis=1,inplace=True)
weather.drop("pressure",axis=1,inplace=True)
weather.drop("FeelsLikeC",axis=1,inplace=True)

#sunset sunriase processing
sunmoonlist = [ "sunrise","sunset"]
for i in range(0,len(weather)):
    for sunMoon in sunmoonlist:
        time = weather[sunMoon][i].split(' ')[1]
        weather[sunMoon][i] = float(weather[sunMoon][i].split(':')[0])
        if(time == "PM"):
            weather[sunMoon][i] = float(weather[sunMoon][i] + 12)
#moonset moonriase processing
# sunmoonlist = [ "moonrise","moonset"]
# for i in range(0,len(weather)):
#     for sunMoon in sunmoonlist:
#         #print(weather[sunMoon][i])
#         if(weather[sunMoon][i][0].isdigit()==False):
#             weather[sunMoon][i]=-1
#             continue
#         time = weather[sunMoon][i].split(' ')[1]
#         weather[sunMoon][i] = float(weather[sunMoon][i].split(':')[0])
#         if(time == "PM"):
#             weather[sunMoon][i] = float(weather[sunMoon][i] + 12)

weather["year"] = pd.to_numeric(weather["year"])
weather["month"] = pd.to_numeric(weather["month"])
weather["day"] = pd.to_numeric(weather["day"])
weather["hour"] = pd.to_numeric(weather["hour"])
weather["sunrise"] = pd.to_numeric(weather["sunrise"])
weather["sunset"] = pd.to_numeric(weather["sunset"])
# weather["moonrise"] = pd.to_numeric(weather["moonrise"])
# weather["moonset"] = pd.to_numeric(weather["moonset"])

weatherName = weather.columns.values.tolist()


In [129]:
data = pd.read_csv('train.csv')
print(data.shape)

(14006, 3)


In [130]:
#data preprocessing
from pandas import to_datetime
data['newdate'] = data['date'].map(lambda x:x.split(' ')[0])
data.drop(columns=['id'],inplace=True, axis=1)
data['date'] = data['date'].map(lambda x:x.split(':')[0])

for i in range(0,len(data)):
    
    data['date'][i] = data['date'][i].replace(" ","/")
    
df_dt=to_datetime(data.date,format="%d/%m/%Y/%H")

/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [131]:
#data preprocessing
#feature engeieering
data['weekday'] = df_dt.dt.weekday
data['year'] = df_dt.dt.year
data['month'] = df_dt.dt.month
data['day'] = df_dt.dt.day
data['hour'] = df_dt.dt.hour
data.drop(columns=['date'],inplace=True, axis=1)

data['weekend0'] = df_dt.dt.weekday
data['weekend1'] = df_dt.dt.weekday

for i in range(1,3):
    string = 'dayHour' + str(i)
    data[string] = df_dt.dt.weekday

data['ithDay'] = df_dt.dt.weekday
data['ithWeek'] = df_dt.dt.weekday

data['season1'] = df_dt.dt.weekday
data['season2'] = df_dt.dt.weekday
data['season3'] = df_dt.dt.weekday
data['season4'] = df_dt.dt.weekday

# add weather related features
# data = pd.concat([data, weather], axis=1, join='inner')
# _, i = np.unique(data.columns, return_index=True)
# data = data.iloc[:, i]


#Add weekend feature
for i in range(0,len(data)):
    if(data["weekday"][i] >=5):
        if(data["weekday"][i] == 5):
            data["weekend0"][i]=1
            data["weekend1"][i]=0
        else:
            data["weekend1"][i]=1
            data["weekend0"][i]=0
    else:
        data["weekend1"][i]=0
        data["weekend0"][i]=0

#Add tiem period features    
for i in range(0,len(data)):
    data['dayHour1'][i]=1 if (data["hour"][i] >= 7 and data["hour"][i] <=19) else 0
    data['dayHour2'][i]=1 if (data["hour"][i] < 7 or data["hour"][i] >19) else 0

#add holiday features
import datetime
data['holiday'] = df_dt.dt.weekday
for n in range(0,len(data)):
    if(data["year"][n] == 2017):
        for i in range(0,len(ho2017)):
            if( int(data["month"][n]) == int(re.split(r'\D+',ho2017[i])[0]) and int(data["day"][n]) == int(re.split(r'\D+',ho2017[i])[1])):
                data['holiday'][n]=1
                break
            else:
                data['holiday'][n]=0
    else:
        for i in range(0,len(ho2018)):
            if( int(data["month"][n]) == int(re.split(r'\D+',ho2018[i])[0]) and int(data["day"][n]) == int(re.split(r'\D+',ho2018[i])[1])):
                data['holiday'][n]=1
                break
            else:
                data['holiday'][n]=0
                
#add ith day, ith week, and season features
for i in range(0,len(data)):
    data['season1'][i]=0
    data['season2'][i]=0
    data['season3'][i]=0
    data['season4'][i]=0
    
for i in range(0,len(data)):
    sday = datetime.date(data['year'][i], data['month'][i], data['day'][i])

    count = sday - datetime.date(sday.year - 1, 12 ,31) # 减去上一年最后一天
    
    #days shift one,obervered by visulization data
    if(data['year'][i]==2017):
        data['ithDay'][i] = count.days+1
    else:
        data['ithDay'][i] = count.days      

    data['ithWeek'][i] = datetime.date(data['year'][i], data['month'][i], data['day'][i]).isocalendar()[1]
    if((data['year'][i] == 2017) and (data['month'][i] == 1) and (data['day'][i] == 1)):
        data['ithWeek'][i]=0
    
    if(data['month'][i]>=1 and data['month'][i]<=3):
        data['season1'][i]=1
    elif(data['month'][i]>=4 and data['month'][i]<=6):
        data['season2'][i]=1
    elif(data['month'][i]>=7 and data['month'][i]<=9):
        data['season3'][i]=1
    elif(data['month'][i]>=10 and data['month'][i]<=12):
        data['season4'][i]=1
    
#add time(month) period features
data['mon0'] = df_dt.dt.weekday
data['mon1'] = df_dt.dt.weekday
data['mon2'] = df_dt.dt.weekday
for i in range(0,len(data)):
    data['mon0'][i] = 0
    data['mon1'][i] = 0
    data['mon2'][i] = 0
    
    if(data['month'][i]>=0 and data['month'][i]<=9):
        data['mon0'][i] = 1
    elif(data['month'][i]>=10 and data['month'][i]<=19):
        data['mon1'][i] = 1
    else:
        data['mon2'][i] = 1

#add timestamp, used for computing some features in test dataset
data['ts'] = df_dt.dt.weekday
import time
import datetime
for i in range(0,len(data)):
    temp=str(data["year"][i])+"-"+str(data["month"][i])+"-"+str(data["day"][i])+"-"+str(data["hour"][i])
    dt = datetime.datetime.strptime(temp, '%Y-%m-%d-%H')
    
    data['ts'][i]=time.mktime(dt.timetuple())
    
#add avg/min/max/median/std/IQR/cof speed in last 24 recorded data(24 hours expected, but lack of interval data)

data['max'] = df_dt.dt.weekday
data['min'] = df_dt.dt.weekday
data['avgSpeed'] = df_dt.dt.weekday
data['std'] = df_dt.dt.weekday
data['median'] = df_dt.dt.weekday
data['iqr'] = df_dt.dt.weekday
####
data['cv'] = df_dt.dt.weekday
####
timeDurarion_ = 24
for i in range(0,len(data)):
    forSTD = np.zeros(timeDurarion_)
    sdaytime = datetime.datetime(data['year'][i], data['month'][i], data['day'][i],data['hour'][i])
    sumS = 0
    maxS = -1
    minS = 1000
    timeDurarion = timeDurarion_
    for n in range(1,timeDurarion+1):
        if(i <= timeDurarion_-1): #testset里面删掉！
            continue
        forSTD[n-1]=data['speed'][i-n]
        sumS = sumS + data['speed'][i-n]
        if(data['speed'][i-n] > maxS):
            maxS = data['speed'][i-n]
        if(data['speed'][i-n] < minS):
            minS = data['speed'][i-n]

    data['avgSpeed'][i] = sumS/timeDurarion
    data['max'][i] = maxS
    data['min'][i] = minS
    data['std'][i] = np.std(forSTD)
    data['median'][i] = np.median(forSTD)
    ######
    mean=np.mean(forSTD) 
    std=np.std(forSTD,ddof=0) 
    data['cv'][i]=(std/mean*100).astype(int) #discritization
    #######
    q75, q25 = np.percentile(forSTD, [75 ,25])
    data['iqr'][i] = q75 - q25
    
#first 24 data get no last 24 hours information
for i in range(0,24):
    if(i <= timeDurarion_-1): 
        data['avgSpeed'][i] = data['avgSpeed'][24] 
        data['max'][i] = data['max'][24]
        data['min'][i] = data['min'][24]
        data['std'][i] = data['std'][24]
        data['median'][i] = data['median'][24]
        data['iqr'][i] = data['iqr'][24]
        ####
        data['cv'][i] = data['cv'][24]
        ####

/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/pytho

/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/py

In [132]:
#orgData used for test dataset preprocessing
orgData = data
data = data.drop('newdate',axis = 1)
data = data.drop('ts',axis = 1)
orgData.head()

,speed,newdate,weekday,year,month,day,hour,weekend0,weekend1,dayHour1,dayHour2,ithDay,ithWeek,season1,season2,season3,season4,holiday,mon0,mon1,mon2,ts,max,min,avgSpeed,std,median,iqr,cv
0,43.002930,1/1/2017,6,2017,1,1,0,0,1,0,1,2,0,1,0,0,0,0,1,0,0,1483200000,49,38,44,2,44,3,6
1,46.118696,1/1/2017,6,2017,1,1,1,0,1,0,1,2,0,1,0,0,0,0,1,0,0,1483203600,49,38,44,2,44,3,6
2,44.294158,1/1/2017,6,2017,1,1,2,0,1,0,1,2,0,1,0,0,0,0,1,0,0,1483207200,49,38,44,2,44,3,6
3,41.067468,1/1/2017,6,2017,1,1,3,0,1,0,1,2,0,1,0,0,0,0,1,0,0,1483210800,49,38,44,2,44,3,6
4,46.448653,1/1/2017,6,2017,1,1,4,0,1,0,1,2,0,1,0,0,0,0,1,0,0,1483214400,49,38,44,2,44,3,6


In [133]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import sklearn.metrics
# get data's features and labels
X_train = data.drop('speed',axis = 1)
y_train = data.speed

In [134]:
from xgboost.sklearn import XGBRegressor
# parameter tuning
# xgb1 = XGBRegressor()
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               #'objective':['reg:linear'],
#               'learning_rate': [0.001,.03, 0.05,.01,.1], #so called `eta` value
#               'max_depth': [5,10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#               'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
#               #'silent': [1],
#               'subsample': [0.6, 0.7, 0.8, 0.9],
#               'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
#               'n_estimators': [400,600,800,1000,2000,3000],
#               'gamma':[0.1,0.3,0.5,0.6],
#               'reg_alpha': [0.05, 0.1, 1, 2], 
#               'reg_lambda': [0.05, 0.1, 1, 2]}

# parameter tuning
# clf = GridSearchCV(xgb1,
#                         parameters,
#                         cv = 5,
#                         n_jobs = 5,
#                         verbose=True,scoring = "neg_mean_squared_error")
# clf.fit(X_train,y_train)
# print(clf.best_estimator_)

#apply best parameter after tuning
final_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.3, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=100,
             min_child_weight=3, monotone_constraints='()',
             n_estimators=3000, n_jobs=4, nthread=4, num_parallel_tree=1,
             random_state=0, reg_alpha=2, reg_lambda=1, scale_pos_weight=1,
             subsample=0.7, tree_method='exact', validate_parameters=1,
             verbosity=None)
final_model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.3, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=100,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=3000, n_jobs=4, nthread=4, num_parallel_tree=1,
             random_state=0, reg_alpha=2, reg_lambda=1, scale_pos_weight=1,
             subsample=0.7, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [135]:
print(final_model.feature_importances_)

[1.5439044e-02 5.2742247e-04 4.8172212e-04 4.3530957e-04 3.7801834e-03
 3.7555969e-03 1.3090260e-01 4.3405670e-01 3.7743336e-01 5.8811373e-04
 6.2073447e-04 9.3697949e-04 4.4506000e-04 5.5777997e-04 6.7635346e-04
 2.5525643e-02 1.4618746e-04 3.4581084e-04 0.0000000e+00 3.7602158e-04
 5.4326933e-04 4.8734449e-04 6.0874416e-04 4.4865749e-04 3.9742223e-04
 4.8386888e-04]


In [136]:
X_train.head(1000)

,weekday,year,month,day,hour,weekend0,weekend1,dayHour1,dayHour2,ithDay,ithWeek,season1,season2,season3,season4,holiday,mon0,mon1,mon2,max,min,avgSpeed,std,median,iqr,cv
0,6,2017,1,1,0,0,1,0,1,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
1,6,2017,1,1,1,0,1,0,1,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
2,6,2017,1,1,2,0,1,0,1,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
3,6,2017,1,1,3,0,1,0,1,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
4,6,2017,1,1,4,0,1,0,1,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
5,6,2017,1,1,5,0,1,0,1,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
6,6,2017,1,1,6,0,1,0,1,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
7,6,2017,1,1,7,0,1,1,0,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
8,6,2017,1,1,8,0,1,1,0,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6
9,6,2017,1,1,9,0,1,1,0,2,0,1,0,0,0,0,1,0,0,49,38,44,2,44,3,6


In [137]:
#test set data preprocessiong
data2 = pd.read_csv('test.csv')
data = data2

#data preprocessing
#feature engeieering
data['newdate'] = data['date'].map(lambda x:x.split(' ')[0])
data.drop(columns=['id'],inplace=True, axis=1)
data['date'] = data['date'].map(lambda x:x.split(':')[0])

for i in range(0,len(data)):
    
    data['date'][i] = data['date'][i].replace(" ","/")

df_dt=to_datetime(data.date,format="%d/%m/%Y/%H")

# add weather features
# data = pd.concat([data, weather], axis=1, join='inner')
# _, i = np.unique(data.columns, return_index=True)
# data = data.iloc[:, i]

#data preprocessing
data['weekday'] = df_dt.dt.weekday
data['year'] = df_dt.dt.year
data['month'] = df_dt.dt.month
data['day'] = df_dt.dt.day
data['hour'] = df_dt.dt.hour
data.drop(columns=['date'],inplace=True, axis=1)


# add weekend feature
data['weekend0'] = df_dt.dt.weekday
data['weekend1'] = df_dt.dt.weekday

for i in range(0,len(data)):
    if(data["weekday"][i] >=5):
        if(data["weekday"][i] == 5):
            data["weekend0"][i]=1
            data["weekend1"][i]=0
        else:
            data["weekend1"][i]=1
            data["weekend0"][i]=0
    else:
        data["weekend1"][i]=0
        data["weekend0"][i]=0

#add time period features
for i in range(1,3):
    string = 'dayHour' + str(i)
    data[string] = df_dt.dt.weekday
for i in range(0,len(data)):
    data['dayHour1'][i]=1 if (data["hour"][i] >= 7 and data["hour"][i] <=19) else 0
    data['dayHour2'][i]=1 if (data["hour"][i] < 7 or data["hour"][i] >19) else 0


# add holiday features
import datetime
data['holiday'] = df_dt.dt.weekday
for n in range(0,len(data)):
    if(data["year"][n] == 2017):
        for i in range(0,len(ho2017)):
            if( int(data["month"][n]) == int(re.split(r'\D+',ho2017[i])[0]) and int(data["day"][n]) == int(re.split(r'\D+',ho2017[i])[1])):
                data['holiday'][n]=1
                break
            else:
                data['holiday'][n]=0
    else:
        for i in range(0,len(ho2018)):
            if( int(data["month"][n]) == int(re.split(r'\D+',ho2018[i])[0]) and int(data["day"][n]) == int(re.split(r'\D+',ho2018[i])[1])):
                data['holiday'][n]=1
                break
            else:
                data['holiday'][n]=0
                
#add ith day, ith week, and season features
data['season1'] = df_dt.dt.weekday
data['season2'] = df_dt.dt.weekday
data['season3'] = df_dt.dt.weekday
data['season4'] = df_dt.dt.weekday
data['ithDay'] = df_dt.dt.weekday
data['ithWeek'] = df_dt.dt.weekday
for i in range(0,len(data)):
    data['season1'][i]=0
    data['season2'][i]=0
    data['season3'][i]=0
    data['season4'][i]=0
    
for i in range(0,len(data)):
    sday = datetime.date(data['year'][i], data['month'][i], data['day'][i])

    count = sday - datetime.date(sday.year - 1, 12 ,31) # 减去上一年最后一天
    
    #days shift one
    if(data['year'][i]==2017):
        data['ithDay'][i] = count.days+1
    else:
        data['ithDay'][i] = count.days      

    data['ithWeek'][i] = datetime.date(data['year'][i], data['month'][i], data['day'][i]).isocalendar()[1]
    if((data['year'][i] == 2017) and (data['month'][i] == 1) and (data['day'][i] == 1)):
        data['ithWeek'][i]=0
    
    if(data['month'][i]>=1 and data['month'][i]<=3):
        data['season1'][i]=1
    elif(data['month'][i]>=4 and data['month'][i]<=6):
        data['season2'][i]=1
    elif(data['month'][i]>=7 and data['month'][i]<=9):
        data['season3'][i]=1
    elif(data['month'][i]>=10 and data['month'][i]<=12):
        data['season4'][i]=1
    
#add time(month) period features
data['mon0'] = df_dt.dt.weekday
data['mon1'] = df_dt.dt.weekday
data['mon2'] = df_dt.dt.weekday
for i in range(0,len(data)):
    data['mon0'][i] = 0
    data['mon1'][i] = 0
    data['mon2'][i] = 0
    
    if(data['month'][i]>=0 and data['month'][i]<=9):
        data['mon0'][i] = 1
    elif(data['month'][i]>=10 and data['month'][i]<=19):
        data['mon1'][i] = 1
    else:
        data['mon2'][i] = 1
        
#add timestamp, used for computing some features in test dataset
data['ts'] = df_dt.dt.weekday
import time
import datetime
for i in range(0,len(data)):
    temp=str(data["year"][i])+"-"+str(data["month"][i])+"-"+str(data["day"][i])+"-"+str(data["hour"][i])
    dt = datetime.datetime.strptime(temp, '%Y-%m-%d-%H')
    
    data['ts'][i]=time.mktime(dt.timetuple())

    
#add avg/min/max/median/std/IQR/cof speed in last 24 recorded data(24 hours expected, but lack of interval data)

data['max'] = df_dt.dt.weekday
data['min'] = df_dt.dt.weekday
data['avgSpeed'] = df_dt.dt.weekday
data['std'] = df_dt.dt.weekday
data['median'] = df_dt.dt.weekday
data['iqr'] = df_dt.dt.weekday
###
data['cv'] = df_dt.dt.weekday
###
timeDurarion_ = 24
for i in range(0,len(data)):
    #initilize for last 3 test data (boundary value)
    if(i==3501 or i==3502 or i==3503):
        data['avgSpeed'][i] = data['avgSpeed'][3500]
        data['max'][i] = data['max'][3500]
        data['min'][i] = data['min'][3500]
        data['std'][i] = data['std'][3500]
        data['median'][i] = data['median'][3500]
        data['iqr'][i] = data['iqr'][3500]
        ###
        data['cv'][i] = data['cv'][3500]
        ###
        continue
    #find the one cloest to current time in test data from train data using timestamp, to calculate avg,std,min...    
    flag = 0
    while(1):
        temp = 14832000000
        #start from year 2018 is enough
        for n in range(8600,len(orgData)):
            temp_ = data['ts'][i]-orgData['ts'][n]
            if(temp_<=0):
                flag=1
                break
            if(temp_<temp):
                orgDataUsed = n
                temp = temp_
        if(flag==1):
            break  

    forSTD = np.zeros(timeDurarion_)
    sumS = 0
    maxS = -1
    minS = 1000
    timeDurarion = timeDurarion_
    for n in range(0,timeDurarion):
        forSTD[n-1]=orgData['speed'][orgDataUsed-n]
        sumS = sumS + orgData['speed'][orgDataUsed-n]
        if(orgData['speed'][orgDataUsed-n] > maxS):
            maxS = orgData['speed'][orgDataUsed-n]
        if(orgData['speed'][orgDataUsed-n] < minS):
            minS = orgData['speed'][orgDataUsed-n]

    data['avgSpeed'][i] = sumS/timeDurarion
    data['max'][i] = maxS
    data['min'][i] = minS
    data['std'][i] = np.std(forSTD)
    data['median'][i] = np.median(forSTD)
    q75, q25 = np.percentile(forSTD, [75 ,25])
    data['iqr'][i] = q75 - q25
    ###
    mean=np.mean(forSTD) 
    std=np.std(forSTD,ddof=0) 
    data['cv'][i]=(std/mean*100).astype(int) #discritization
    ###

data.drop(columns=['newdate'],inplace=True, axis=1)
data.drop(columns=['ts'],inplace=True, axis=1)

/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/pytho

/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xingzhenghao/opt/anaconda3/lib/py

In [138]:
data.head()

,weekday,year,month,day,hour,weekend0,weekend1,dayHour1,dayHour2,holiday,season1,season2,season3,season4,ithDay,ithWeek,mon0,mon1,mon2,max,min,avgSpeed,std,median,iqr,cv
0,0,2018,1,1,2,0,0,0,1,1,1,0,0,0,1,1,1,0,0,48,38,44,2,44,3,6
1,0,2018,1,1,5,0,0,0,1,1,1,0,0,0,1,1,1,0,0,50,38,44,3,44,3,7
2,0,2018,1,1,7,0,0,1,0,1,1,0,0,0,1,1,1,0,0,50,38,44,3,44,3,7
3,0,2018,1,1,8,0,0,1,0,1,1,0,0,0,1,1,1,0,0,50,38,44,3,44,3,7
4,0,2018,1,1,10,0,0,1,0,1,1,0,0,0,1,1,1,0,0,50,36,43,3,44,3,8


In [139]:
final_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.3, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=100,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=3000, n_jobs=4, nthread=4, num_parallel_tree=1,
             random_state=0, reg_alpha=2, reg_lambda=1, scale_pos_weight=1,
             subsample=0.7, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [140]:
data = data[X_train.columns]
data.head()

,weekday,year,month,day,hour,weekend0,weekend1,dayHour1,dayHour2,ithDay,ithWeek,season1,season2,season3,season4,holiday,mon0,mon1,mon2,max,min,avgSpeed,std,median,iqr,cv
0,0,2018,1,1,2,0,0,0,1,1,1,1,0,0,0,1,1,0,0,48,38,44,2,44,3,6
1,0,2018,1,1,5,0,0,0,1,1,1,1,0,0,0,1,1,0,0,50,38,44,3,44,3,7
2,0,2018,1,1,7,0,0,1,0,1,1,1,0,0,0,1,1,0,0,50,38,44,3,44,3,7
3,0,2018,1,1,8,0,0,1,0,1,1,1,0,0,0,1,1,0,0,50,38,44,3,44,3,7
4,0,2018,1,1,10,0,0,1,0,1,1,1,0,0,0,1,1,0,0,50,36,43,3,44,3,8


In [141]:
test_speed = final_model.predict(data)

In [142]:
test_speed[0:10]

array([48.273335, 47.91863 , 39.513943, 29.863686, 40.60672 , 27.648443,
       19.734406, 49.103764, 48.32167 , 32.14679 ], dtype=float32)

In [143]:
testFinal = pd.read_csv('test.csv')
testFinal.head()

for i in range(0,len(testFinal)):
    testFinal['id'][i] = int(testFinal['id'][i])
    testFinal['date'][i] = test_speed[i]
    
testFinal.columns = ['id','speed']
print(testFinal[0:20])

testFinal.to_csv('mytest.csv',encoding='utf-8', columns=['id','speed'], index=False)

/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/xingzhenghao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


    id    speed
0    0  48.2733
1    1  47.9186
2    2  39.5139
3    3  29.8637
4    4  40.6067
5    5  27.6484
6    6  19.7344
7    7  49.1038
8    8  48.3217
9    9  32.1468
10  10  31.3093
11  11  25.6177
12  12  26.4532
13  13  27.0845
14  14   27.927
15  15  40.5224
16  16  44.4249
17  17  47.8863
18  18  49.6909
19  19  48.2079
